# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

## Question 1
This question will walk you through creating your own `kmeans` function.

#### a) What are the steps of `kmeans`?
**Hint**: There are 4 steps/builder functions that you'll need.

Initialize centroids: randomly choose k points (centroids) from the dataset.

Assign clusters: assign each data point to the nearest centroid.

Update centroids: compute new centroids as the mean of all points assigned to each cluster.

Check for convergence — repeat steps 2 and 3 until centroids no longer change significantly or until a maximum number of iterations is reached.

#### b) Create the builder function for step 1.

In [17]:
initialize_centroids <- function(X, k) {
  set.seed(42)
  indices <- sample(1:nrow(X), k)
  centroids <- X[indices, , drop = FALSE]
  return(centroids)
}

#### c) Create the builder function for step 2.

In [18]:
assign_clusters <- function(X, centroids) {
  distances <- as.matrix(dist(rbind(centroids, X)))[1:nrow(centroids), (nrow(centroids)+1):(nrow(centroids)+nrow(X))]
  cluster_assignments <- apply(distances, 2, which.min)
  return(cluster_assignments)
}

#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

In [19]:
update_centroids <- function(X, clusters, k) {
  new_centroids <- matrix(NA, nrow = k, ncol = ncol(X))
  for (i in 1:k) {
    cluster_points <- X[clusters == i, , drop = FALSE]
    if (nrow(cluster_points) > 0) {
      new_centroids[i, ] <- colMeans(cluster_points)
    } else {
      new_centroids[i, ] <- X[sample(1:nrow(X), 1), ]
    }
  }
  return(new_centroids)
}


#### e) Create the builder function for step 4.

In [20]:
has_converged <- function(old_centroids, new_centroids, tol = 1e-4) {
  shifts <- sqrt(rowSums((new_centroids - old_centroids)^2))
  return(all(shifts < tol))
}

#### f) Combine them all into your own `kmeans` function.

In [21]:
my_kmeans <- function(X, k, max_iters = 100, tol = 1e-4) {
  centroids <- initialize_centroids(X, k)
  
  for (iter in 1:max_iters) {
    clusters <- assign_clusters(X, centroids)
    
    new_centroids <- update_centroids(X, clusters, k)
    
    if (has_converged(centroids, new_centroids, tol)) {
      cat("Converged after", iter, "iterations\n")
      break
    }
    
    centroids <- new_centroids
  }
  
  return(list(centroids = centroids, clusters = clusters))
}


## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set. 

In [22]:
library(readr)
voltages_df <- read_csv("voltages_df.csv")

Rows: 900 Columns: 250
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (250): 0, 1.00401606425703, 2.00803212851406, 3.01204819277108, 4.016064...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`. 

In [23]:
voltages_matrix <- as.matrix(voltages_df)
results <- my_kmeans(voltages_matrix, k = 3)
cat("Cluster Labels (results$labels):\n")
print(results$clusters) 
cat("\nCluster Means (results$means):\n")
print(results$centroids)  

Converged after 8 iterations
Cluster Labels (results$labels):
  4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23 
  2   3   3   3   3   1   2   2   3   2   2   2   2   2   3   2   2   2   2   2 
 24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43 
  1   3   3   3   3   2   2   2   2   2   2   2   3   2   2   3   3   2   2   1 
 44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63 
  2   2   2   2   1   2   2   3   2   2   1   2   2   2   2   2   1   2   2   3 
 64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83 
  2   2   3   2   2   2   1   2   2   2   2   3   2   2   3   1   2   2   2   2 
 84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 
  2   2   3   2   2   3   3   3   2   2   2   2   3   2   3   1   2   3   3   2 
104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 
  2   2   3   2   2   2   2   2   2   1   2   2

#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`. 
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [24]:
r_results <- kmeans(voltages_matrix, centers = 3)
cat("Cluster Labels (results$cluster):\n")
print(r_results$cluster)
cat("\nCluster Means (results$centers):\n")
print(r_results$centers)

Cluster Labels (results$cluster):
  [1] 3 3 3 3 3 3 1 1 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 2 2 1 3 2 2 3 3
 [38] 3 2 3 3 3 3 2 3 1 1 3 2 2 3 1 3 3 2 3 3 3 2 3 3 3 3 3 2 2 3 1 3 2 3 3 1 1
 [75] 3 3 3 1 2 1 2 2 3 1 2 3 3 3 3 1 1 2 3 2 3 3 2 3 3 2 1 2 3 2 3 3 2 3 3 3 3
[112] 3 3 2 3 1 3 3 3 3 3 3 3 3 2 3 3 1 3 2 3 3 3 2 3 3 3 3 3 1 3 3 3 1 3 3 3 3
[149] 3 1 3 1 3 3 1 3 2 1 3 3 1 2 3 3 3 2 3 3 3 3 3 2 3 3 2 2 3 2 3 3 1 3 1 3 2
[186] 3 3 3 3 3 3 3 3 3 1 2 3 3 3 3 1 3 2 2 3 3 3 2 3 3 3 1 3 1 1 3 1 2 3 3 3 3
[223] 3 3 3 1 2 2 2 3 3 3 1 3 3 2 2 3 3 1 1 3 1 1 3 2 3 1 1 3 2 3 3 2 3 1 1 3 3
[260] 3 3 3 1 3 3 3 3 1 1 1 3 3 3 3 2 3 2 2 3 3 1 3 1 3 3 3 1 3 3 2 1 3 1 1 3 3
[297] 3 2 2 3 3 1 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 3 3 3 3 3 3 3 1
[334] 3 3 3 2 2 3 3 2 3 1 2 3 1 3 1 3 3 2 3 3 3 3 2 3 3 1 2 3 3 3 3 3 1 3 3 3 3
[371] 3 3 3 3 3 1 2 3 3 3 3 2 2 3 3 2 2 3 3 2 3 3 3 3 3 1 3 3 2 3 3 3 3 3 3 3 3
[408] 1 1 1 3 1 1 3 3 3 3 1 3 3 3 3 3 3 3 2 3 3 3 3 1 2 3 2 3 3 3 3 3 1 2 1 1 3
[445] 

#### d) Are your labels/clusters the same? If not, why? Are your means the same?
The specific cluster labels differ due to random initialization, but the resulting centroids (means) are approximately the same, showing that both implementations successfully identify similar cluster structures.

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.
In a for loop approach, K-Means assigns clusters by iterating through each data point, calculating its distance to all centroids, and assigning it to the nearest one. This process repeats for every observation in the dataset. While this method is simple and intuitive, it requires looping through each point individually, which can be slow when dealing with large datasets.

#### b) Explain the process of vectorizing the code to assign clusters for kmeans.
In a vectorized approach, the code computes all distances between points and centroids simultaneously using matrix operations instead of explicit loops. This leverages R’s internal optimization and allows entire columns or matrices to be processed at once. As a result, cluster assignments are made much faster by finding the minimum distance for each point in a single vectorized step.

#### c) State which (for loops or vectorizing) is more efficient and why.
Vectorization is more efficient because R’s vector and matrix operations are implemented in optimized C code, allowing computations to run much faster than interpreted for loops. For loops process each observation one at a time, while vectorized operations handle all data points simultaneously, reducing computation time and improving scalability.

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?
K-Means fails when clusters are not spherical, are of unequal size or density, or overlap significantly. The algorithm assumes that clusters are roughly circular (or convex) and that each point belongs to exactly one cluster based on its proximity to a centroid. When this assumption is violated—such as with elongated, irregularly shaped, or differently sized clusters—K-Means can incorrectly group points or split natural clusters.

## Question 5
#### What assumption do Guassian mixture models make?
Gaussian Mixture Models assume that the data are generated from a mixture of several Gaussian (normal) distributions, each representing a cluster. Each cluster is defined by its own mean and covariance matrix, allowing for elliptical shapes and variable cluster sizes. This flexibility makes GMMs more general than K-Means, which assumes equal variance and spherical clusters.

## Question 6
#### What assumption does spectral clustering make? Why does this help us?
Spectral clustering assumes that clusters can be identified based on the connectivity or similarity between points rather than just their geometric distance. It uses eigenvalues of a similarity matrix to map data into a lower-dimensional space where the clusters become more clearly separated. This helps reveal complex, non-convex cluster structures that methods like K-Means cannot detect.

## Question 7
#### Define the gap statistic method. What do we use it for?
The gap statistic method compares the within-cluster variation for a given number of clusters with what would be expected under a random, uniform distribution of points. It is used to determine the optimal number of clusters k by identifying the value of k that maximizes the gap between the observed clustering performance and the reference null model, indicating the most meaningful cluster separation.